In [15]:
import os, json, re, math, random, string, unicodedata
from pathlib import Path
import numpy as np
import pandas as pd

# Reproducibilidad
SEED = 42
random.seed(SEED); np.random.seed(SEED)


DATA_DIR   = Path("../data")
REPORTS    = Path("../reports")
MODELS_DIR = Path("../models")
CONF_DIR   = Path("../configs")
DATA_DIR.mkdir(parents=True, exist_ok=True)
REPORTS.mkdir(parents=True, exist_ok=True)
MODELS_DIR.mkdir(parents=True, exist_ok=True)
CONF_DIR.mkdir(parents=True, exist_ok=True)

INPUT_CSV  = DATA_DIR / "processed_clean.csv"     # del EDA
OUT_PROCES = DATA_DIR / "processed.csv"           # con columnas binarias por clase
OUT_CLASSES= DATA_DIR / "classes.txt"             # lista de clases
SPLIT_PATH = DATA_DIR / "splits.json"             # índices de train/val/test
VEC_PATH   = MODELS_DIR / "baseline" / "tfidf.joblib"  
(Path(MODELS_DIR / "baseline")).mkdir(parents=True, exist_ok=True)

print("Config OK")


Config OK


In [16]:
assert INPUT_CSV.exists(), f"No encuentro {INPUT_CSV}. Genera este archivo al final del 01_eda.ipynb."
df = pd.read_csv(INPUT_CSV)
print(df.shape)
df.head(2)


(3563, 4)


,title,abstract,group,text
0,"""Real-world"" data on the efficacy and safety o...",Lenalidomide and dexamethasone (RD) is a stand...,neurological,"""Real-world"" data on the efficacy and safety o..."
1,22-oxacalcitriol suppresses secondary hyperpar...,BACKGROUND: Calcitriol therapy suppresses seru...,hepatorenal,22-oxacalcitriol suppresses secondary hyperpar...


In [17]:
# Validaciones fuertes del esquema
expected_cols = {"title","abstract","group","text"}
missing = expected_cols - set(df.columns)
assert not missing, f"Faltan columnas: {missing}"

assert df["title"].isna().sum()==0, "title tiene NaN"
assert df["abstract"].isna().sum()==0, "abstract tiene NaN"
assert df["group"].isna().sum()==0, "group tiene NaN"
assert len(df) >= 3000, "Tamaño inesperado; revisa el EDA/limpieza"

# Documentar procedencia en un archivo
provenance = {
    "source": str(INPUT_CSV),
    "rows": int(len(df)),
    "columns": df.columns.tolist(),
    "note": "Archivo limpio exportado desde 01_eda.ipynb tras eliminar duplicados y fusionar por title.",
}
with open(REPORTS / "provenance_preprocessing.json", "w", encoding="utf-8") as f:
    json.dump(provenance, f, indent=2, ensure_ascii=False)
provenance


{'source': '..\\data\\processed_clean.csv',
 'rows': 3563,
 'columns': ['title', 'abstract', 'group', 'text'],
 'note': 'Archivo limpio exportado desde 01_eda.ipynb tras eliminar duplicados y fusionar por title.'}

In [18]:
from textwrap import dedent
justificacion = dedent("""
Decisiones de preprocesamiento:
1) Unificamos 'title' + 'abstract' en 'text' (hecho en el EDA) para simplificar la entrada del modelo.
2) Normalizamos texto de forma mínima y segura para datos biomédicos:
   - Minusculización y limpieza de espacios.
   - Conservamos dígitos, guiones y términos clínicos (no removemos agresivamente).
   - Evitamos stemming/lemmatization fuerte para no romper terminología médica.
3) Binarizamos etiquetas multilabel con un listado determinístico de clases.
4) Dividimos train/val/test con estratificación **multilabel** (iterative-stratification) y semilla fija.
5) Generamos TF-IDF aquí si el baseline clásico lo requiere, para ahorrar tiempo en el entrenamiento.
""").strip()
print(justificacion)


Decisiones de preprocesamiento:
1) Unificamos 'title' + 'abstract' en 'text' (hecho en el EDA) para simplificar la entrada del modelo.
2) Normalizamos texto de forma mínima y segura para datos biomédicos:
   - Minusculización y limpieza de espacios.
   - Conservamos dígitos, guiones y términos clínicos (no removemos agresivamente).
   - Evitamos stemming/lemmatization fuerte para no romper terminología médica.
3) Binarizamos etiquetas multilabel con un listado determinístico de clases.
4) Dividimos train/val/test con estratificación **multilabel** (iterative-stratification) y semilla fija.
5) Generamos TF-IDF aquí si el baseline clásico lo requiere, para ahorrar tiempo en el entrenamiento.


In [19]:
# Nota: Normalización minimalista—evita destruir términos como "BRCA1", "TNF-α", etc.
# Convertimos a minúsculas, colapsamos espacios, removemos caracteres invisibles raros.
# Preservamos dígitos y signos relevantes (%, -, /) que aparecen en resúmenes.

def normalize_text(s: str) -> str:
    if not isinstance(s, str):
        s = "" if pd.isna(s) else str(s)
    # Unicode NFKC: normaliza formas de caracteres
    s = unicodedata.normalize("NFKC", s)
    # Minúsculas
    s = s.lower()
    # Reemplazar múltiples espacios y saltos por un espacio
    s = re.sub(r"\s+", " ", s).strip()
    return s

df["text_norm"] = df["text"].apply(normalize_text)

# Comprobación rápida
print(df[["text","text_norm"]].head(1).to_string(index=False, max_colwidth=90))


                                                                                      text                                                                                  text_norm
"Real-world" data on the efficacy and safety of lenalidomide and dexamethasone in patie... "real-world" data on the efficacy and safety of lenalidomide and dexamethasone in patie...


In [20]:
# Listas de etiquetas por fila
labels_series = df["group"].astype(str).apply(lambda s: [t.strip() for t in s.split("|") if t.strip()!=""])

# Conjunto de clases (orden determinista)
classes = sorted({lab for L in labels_series for lab in L})
print("Clases:", classes)

# Matriz binaria Y (n x C)
Y = pd.DataFrame([{c: int(c in L) for c in classes} for L in labels_series], dtype=int)
print(Y.sum().to_dict())  # positivos por clase

# Guardar lista de clases
with open(OUT_CLASSES, "w", encoding="utf-8") as f:
    f.write("\n".join(classes))
print("Guardado:", OUT_CLASSES)


Clases: ['cardiovascular', 'hepatorenal', 'neurological', 'oncological']
{'cardiovascular': 1267, 'hepatorenal': 1091, 'neurological': 1784, 'oncological': 600}
Guardado: ..\data\classes.txt


In [21]:
out = pd.concat([df[["title","abstract","group","text","text_norm"]], Y], axis=1)
out.to_csv(OUT_PROCES, index=False)
print("Guardado:", OUT_PROCES, "| shape:", out.shape)
out.head(2)


Guardado: ..\data\processed.csv | shape: (3563, 9)


,title,abstract,group,text,text_norm,cardiovascular,hepatorenal,neurological,oncological
0,"""Real-world"" data on the efficacy and safety o...",Lenalidomide and dexamethasone (RD) is a stand...,neurological,"""Real-world"" data on the efficacy and safety o...","""real-world"" data on the efficacy and safety o...",0,0,1,0
1,22-oxacalcitriol suppresses secondary hyperpar...,BACKGROUND: Calcitriol therapy suppresses seru...,hepatorenal,22-oxacalcitriol suppresses secondary hyperpar...,22-oxacalcitriol suppresses secondary hyperpar...,0,1,0,0


In [22]:
# pip install iterative-stratification antes de correr
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from sklearn.model_selection import train_test_split

# Estrategia: primero separo TEST (15%), luego CV 5-fold sobre el 85% restante para entrenar y validar
TEST_SIZE = 0.15
N_SPLITS  = 5

indices = np.arange(len(out))
X_dummy  = np.zeros((len(out), 1))  # no se usa, pero requerido
Y_mat    = out[classes].values

# Split holdout test estratificado
idx_trainval, idx_test = next(
    MultilabelStratifiedKFold(n_splits=int(1/TEST_SIZE), shuffle=True, random_state=SEED).split(X_dummy, Y_mat)
)
# Lo anterior crea 1 fold como "test". Ajustamos por si no cae exacto al 15%
idx_trainval = np.array(idx_trainval)
idx_test     = np.array(idx_test)

print("Train+Val:", len(idx_trainval), " | Test:", len(idx_test))

# Ahora CV 5-fold dentro de train/val
mskf = MultilabelStratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)
folds = []
for fold_id, (tr, va) in enumerate(mskf.split(X_dummy[idx_trainval], Y_mat[idx_trainval])):
    folds.append({
        "fold": fold_id,
        "train_idx": idx_trainval[tr].tolist(),
        "val_idx":   idx_trainval[va].tolist()
    })

splits = {
    "seed": SEED,
    "test_idx": idx_test.tolist(),
    "folds": folds,
    "classes": classes
}
with open(SPLIT_PATH, "w", encoding="utf-8") as f:
    json.dump(splits, f, indent=2)
print("Guardado:", SPLIT_PATH)
len(idx_test), [len(f["train_idx"]) for f in folds], [len(f["val_idx"]) for f in folds]


Train+Val: 2963  | Test: 600
Guardado: ..\data\splits.json


(600, [2368, 2373, 2368, 2369, 2374], [595, 590, 595, 594, 589])

In [23]:
# Usamos MultilabelStratifiedKFold para mantener las proporciones por clase en cada split. 
# Primero separamos un test estable (evaluación final), y luego hacemos 5 folds para ajustar el modelo y umbrales.

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib

tfidf = TfidfVectorizer(
    ngram_range=(1,2),
    max_features=50000,
    min_df=3,
    sublinear_tf=True,     
    lowercase=False        
)

X_tfidf = tfidf.fit_transform(out["text_norm"].astype(str).values)
print("TF-IDF shape:", X_tfidf.shape)

joblib.dump(tfidf, VEC_PATH)
print("Guardado vectorizador:", VEC_PATH)


TF-IDF shape: (3563, 23621)
Guardado vectorizador: ..\models\baseline\tfidf.joblib


In [25]:
# TF-IDF (1–2-gramas, min_df=3, sublinear_tf=True) rinde fuerte como baseline en textos clínicos. 
# Guardamos el vectorizador para reutilizar.

In [26]:
# Pequeña muestra anónima para que terceros prueben el pipeline sin datos completos
sample = out.sample(n=min(200, len(out)), random_state=SEED)  # 200 filas
sample.to_csv(DATA_DIR / "sample.csv", index=False)
print("Guardado:", DATA_DIR / "sample.csv")


Guardado: ..\data\sample.csv


In [27]:
config = {
    "seed": SEED,
    "input": str(INPUT_CSV),
    "outputs": {
        "processed_csv": str(OUT_PROCES),
        "classes_txt": str(OUT_CLASSES),
        "splits_json": str(SPLIT_PATH),
        "tfidf_vectorizer": str(VEC_PATH)
    },
    "preprocessing": {
        "normalization": "unicode NFKC, lowercase, trim spaces",
        "field_used": "text_norm",
        "keep_digits": True,
        "avoid_aggressive_stemming": True
    },
    "labels": classes,
    "split": {
        "test_size_approx": 0.15,
        "n_splits_cv": 5,
        "stratification": "MultilabelStratifiedKFold"
    },
    "tfidf": {
        "enabled": True,
        "ngram_range": [1,2],
        "max_features": 50000,
        "min_df": 3,
        "sublinear_tf": True,
        "lowercase": False
    }
}
with open(CONF_DIR / "preprocessing_config.json", "w", encoding="utf-8") as f:
    json.dump(config, f, indent=2, ensure_ascii=False)
print("Guardado:", CONF_DIR / "preprocessing_config.json")


Guardado: ..\configs\preprocessing_config.json


In [28]:
report = {
    "rows": int(len(out)),
    "cols": out.shape[1],
    "classes": classes,
    "positives_per_class": {c: int(out[c].sum()) for c in classes},
    "has_text_norm": "text_norm" in out.columns,
    "files": {
        "processed_csv": str(OUT_PROCES.exists()),
        "classes_txt": str(OUT_CLASSES.exists()),
        "splits_json": str(SPLIT_PATH.exists()),
        "tfidf_vectorizer": str(VEC_PATH.exists())
    }
}
report


{'rows': 3563,
 'cols': 9,
 'classes': ['cardiovascular', 'hepatorenal', 'neurological', 'oncological'],
 'positives_per_class': {'cardiovascular': 1267,
  'hepatorenal': 1091,
  'neurological': 1784,
  'oncological': 600},
 'has_text_norm': True,
 'files': {'processed_csv': 'True',
  'classes_txt': 'True',
  'splits_json': 'True',
  'tfidf_vectorizer': 'True'}}